In [268]:
import pandas as pd

In [269]:
UK_to_ISCO_1to1s = pd.read_excel('UK_to_International.xls', header=0, sheetname='1to1s')
UK_to_ISCO_1to2s = pd.read_excel('UK_to_International.xls', header=0, sheetname='1to2 50-50').fillna(method='ffill')
UK_to_ISCO_1to2s_40 = pd.read_excel('UK_to_International.xls', header=0,sheetname='1to2 40-60').fillna(method='ffill')

# Drop the annoying two-to-one entries from UK_to_ISCO_1to1s
UK_null_ix = UK_to_ISCO_1to1s['SOC\n2010'].isnull().nonzero()[0]
UK_null_ix = np.concatenate([UK_null_ix - 1, UK_null_ix])
UK_to_ISCO_1to1s.drop(UK_null_ix, inplace=True)

# Naively merge UK_to_ISCO dataframes
UK_to_ISCO = pd.concat([UK_to_ISCO_1to1s, UK_to_ISCO_1to2s, UK_to_ISCO_1to2s_40], ignore_index=True)

# UK_to_ISCO.rename(columns = {'SOC\n2010': 'SOC2010'}, inplace=True)
UK_to_ISCO.columns = [['Universal'] * UK_to_ISCO.columns.size, UK_to_ISCO.columns]
UK_to_ISCO.columns.names = ['Region', 'Features']

UK_to_ISCO['Universal', 'SOC2010'] = UK_to_ISCO[u'Universal', u'SOC\n2010'].astype(int)
UK_to_ISCO.drop('SOC\n2010', axis=1, level=1, inplace=True)
UK_to_ISCO.head() 

# UK_to_ISCO.columns.levels[1]

Region   Universal                                           \
Features    ISCO08                 ISCO08 Unit Group Titles   
0             1120  Managing directors and chief executives   
1             1111                              Legislators   
2             1321                   Manufacturing managers   
3             1323                    Construction managers   
4             1322                          Mining managers   

Region                                                                          
Features                          SOC2010 Unit Group Titles Unnamed: 4 SOC2010  
0                     Chief executives and senior officials        NaN    1115  
1                      Elected officers and representatives        NaN    1116  
2         Production managers and directors in manufactu...        NaN    1121  
3         Production managers and directors in construction        NaN    1122  
4         Production managers and directors in mining an...        NaN    1123

In [270]:
##########################################################################
# Employment
##########################################################################
# Load in regional employment data
UK_emp = pd.read_excel('empbysocandqualificationsjd13_tcm77-368924.xls',
                       header=7, skip_rows=1, skip_footer=11, na_values=['*', '-'],
                       parse_cols=range(1, 16), sheetname='Emp by occupations and regions')

n_reg = UK_emp.columns.size
# Annoyingly, the default index read contains verbose occupation descriptions
UK_emp['Universal'.decode('utf-8')] = UK_emp.index
UK_emp.index = pd.Index(np.arange(UK_emp.index.size))
UK_emp.drop([0], inplace=True)
UK_emp['Universal'] = UK_emp['Universal'].map(lambda s: int(s[:4]))
UK_emp.fillna(0, inplace=True)

# Do some hierachical indexing
UK_emp.columns = [UK_emp.columns, ['Employment'] * n_reg + ['SOC2010']]
UK_emp.columns.names = ['Region', 'Features']

UK_emp.head()


Region,North East,North West,Merseyside,Yorkshire & Humberside,East Midlands,West Midlands,Eastern,London,South East,South West,Wales,Scotland,Northern Ireland,Total,Universal
Features,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,SOC2010
1,621,4241,827,2556,4460,3180,6491,19992,8604,5097,1985,2771,1355,62180,1115
2,269,1465,0,0,1098,0,0,0,1557,1460,527,855,0,8885,1116
3,11260,31887,2704,24008,26571,31498,34087,16631,35510,25140,10057,19835,7609,276797,1121
4,6956,10549,3079,13246,13447,14702,20974,17148,23870,12476,5217,9606,4566,155836,1122
5,519,0,0,1243,0,861,0,0,1238,1689,903,5697,0,14608,1123


In [271]:
# print UK_emp.columns.values[-1]
# print UK_to_ISCO.columns.values[2]

In [274]:
# Merge in employment data
UK_to_ISCO = pd.merge(UK_to_ISCO, UK_emp, how='inner', on=[('Universal', 'SOC2010')])
UK_to_ISCO.head()


Region   Universal                                           \
Features    ISCO08                 ISCO08 Unit Group Titles   
0             1120  Managing directors and chief executives   
1             1111                              Legislators   
2             1321                   Manufacturing managers   
3             1323                    Construction managers   
4             1322                          Mining managers   

Region                                                                  \
Features                          SOC2010 Unit Group Titles Unnamed: 4   
0                     Chief executives and senior officials        NaN   
1                      Elected officers and representatives        NaN   
2         Production managers and directors in manufactu...        NaN   
3         Production managers and directors in construction        NaN   
4         Production managers and directors in mining an...        NaN   

Region           North East_x North West_x Merseyside_x  \
Features SOC2010   Employment   Employment   Employment   
0           1115          621         4241          827   
1           1116          269         1465            0   
2           1121        11260        31887         2704   
3           1122         6956        10549         3079   
4           1123          519            0            0   

Region   Yorkshire & Humberside_x East Midlands_x    ...     East Midlands_y  \
Features               Employment      Employment    ...          Employment   
0                            2556            4460    ...                4460   
1                               0            1098    ...                1098   
2                           24008           26571    ...               26571   
3                           13246           13447    ...               13447   
4                            1243               0    ...                   0   

Region   West Midlands_y  Eastern_y   London_y South East_y South West_y  \
Features      Employment Employment Employment   Employment   Employment   
0                   3180       6491      19992         8604         5097   
1                      0          0          0         1557         1460   
2                  31498      34087      16631        35510        25140   
3                  14702      20974      17148        23870        12476   
4                    861          0          0         1238         1689   

Region      Wales_y Scotland_y Northern Ireland_y    Total_y  
Features Employment Employment         Employment Employment  
0              1985       2771               1355      62180  
1               527        855                  0       8885  
2             10057      19835               7609     276797  
3              5217       9606               4566     155836  
4               903       5697                  0      14608  

[5 rows x 33 columns]

In [275]:
##########################################################################
# Education
##########################################################################

# Load in regional education data
UK_edu = pd.read_excel('empbysocandqualificationsjd13_tcm77-368924.xls',
                       header=8,
                       skip_footer=11,
                       na_values=['*', '-'],
                       parse_cols=range(1, 30),
                       sheetname='Emp by occupations,regions qual')
# Annoyingly, the default index read contains verbose occupation
# descriptions
UK_edu['Universal'] = UK_edu.index
UK_edu.index = pd.Index(np.arange(UK_edu.index.size))
UK_edu.drop([0, 1], inplace=True)
UK_edu['Universal'] = UK_edu['Universal'].map(lambda s: int(s[:4]))
UK_edu.fillna(0, inplace=True)
# Do some hierachical indexing
# Exclude Universal column
doubled_regions = np.array(UK_edu.columns[:(2 * n_reg)])
# fill in missing column names
doubled_regions[1::2] = doubled_regions[0::2]
UK_edu.columns = [
    np.append(doubled_regions, 'Universal'),
    ['Higher Degree', 'Ordinary degree or equivalent'] *
    n_reg + ['SOC2010']
]
UK_edu.columns.names = ['Region', 'Features']

# Load in further regional education data
UK_edu_2 = pd.read_excel('JD13 foundation degree.xls',
                         header=7,
                         skip_footer=10,
                         na_values=['*', '-'],
                         parse_cols=list(range(1, 16)),
                         sheetname='Emp by occupations and quals')
# Annoyingly, the default index read contains verbose occupation
# descriptions
UK_edu_2['Universal'] = UK_edu_2.index
UK_edu_2.index = pd.Index(np.arange(UK_edu_2.index.size))
UK_edu_2.drop([0, 1], inplace=True)
UK_edu_2['Universal'] = UK_edu_2['Universal'].map(lambda s: int(s[:4]))
UK_edu_2.fillna(0, inplace=True)
# Do some hierachical indexing
# Exclude Universal column
regions = np.array(UK_edu_2.columns[:n_reg])
# fill in missing column names
UK_edu_2.columns = [
    np.append(regions, 'Universal'),
    ['First degree/foundation degree'] * n_reg + ['SOC2010']
]
UK_edu_2.columns.names = ['Region', 'Features']

UK_edu.head()



Region      North East                                  North West  \
Features Higher Degree Ordinary degree or equivalent Higher Degree   
2                    0                             0             0   
3                    0                           269             0   
4                 1432                          1955          5444   
5                    0                          1063          1198   
6                    0                             0             0   

Region                                    Merseyside  \
Features Ordinary degree or equivalent Higher Degree   
2                                 1471             0   
3                                    0             0   
4                                 6592             0   
5                                 2642             0   
6                                    0             0   

Region                                 Yorkshire & Humberside  \
Features Ordinary degree or equivalent          Higher Degree   
2                                    0                    931   
3                                    0                      0   
4                                  645                   2545   
5                                 1496                      0   
6                                    0                      0   

Region                                 East Midlands  \
Features Ordinary degree or equivalent Higher Degree   
2                                  786          1706   
3                                    0             0   
4                                 5342          2689   
5                                 2422          1225   
6                                    0             0   

Region                                    ...     \
Features Ordinary degree or equivalent    ...      
2                                 2197    ...      
3                                    0    ...      
4                                 6474    ...      
5                                 2097    ...      
6                                    0    ...      

Region                      South West         Wales  \
Features Ordinary degree or equivalent Higher Degree   
2                                 2630           542   
3                                  578             0   
4                                 5467          1241   
5                                 2474           216   
6                                  803             0   

Region                                      Scotland  \
Features Ordinary degree or equivalent Higher Degree   
2                                  837           541   
3                                    0             0   
4                                 2826          2102   
5                                 1247           243   
6                                    0           573   

Region                                 Northern Ireland  \
Features Ordinary degree or equivalent    Higher Degree   
2                                 1119                0   
3                                  320                0   
4                                 4925                0   
5                                 2365                0   
6                                 1539                0   

Region                                         Total  \
Features Ordinary degree or equivalent Higher Degree   
2                                    0         15757   
3                                    0          1046   
4                                 1231         30543   
5                                  957          8876   
6                                    0          2275   

Region                                 Universal  
Features Ordinary degree or equivalent   SOC2010  
2                                30017      1115  
3                                 3614      1116  
4                                63717      1121  
5                                35822      1122  
6  

In [277]:
# Merge in education data

UK_to_ISCO = pd.merge(UK_to_ISCO, UK_edu, how='inner')
UK_to_ISCO = pd.merge(UK_to_ISCO, UK_edu_2, how='inner')

In [279]:
##########################################################################
# Income
##########################################################################

UK_income = pd.read_excel(
    ('Work Region Occupation SOC10 (4) Table 15.7a   '
     + 'Annual pay - Gross 2013.xls'),
    sheetname='All',
    header=4,
    parse_cols=[0, 1, 3, 5],
    skip_footer=6,
    na_values=['x']
)
#  Hold off on hierarchical indexing to make life a bit easier
UK_income.columns = (
    ['Description', 'SOC2010', 'Median Income', 'Mean Income']
)

# We need to do some processing on the Description column to extract the
# regions

def find_region(s):
    """
    return a region, if found in the input string s, or else return 'Total'
    """
    # This to allow for the two different names for Eastern
    # s += 'ern'
    # Actually, no, don't: Eastern contains data aggregated at the wrong
    # level
    for region in regions:
        # the odd [:9] slice below is to allow for the two different names
        # for yorkshire: in one dataset
        if region[:9] in s:
            return region
    else:
        return 'Total'

UK_income['Region'] = (
    UK_income['Description'].map(find_region)
)
UK_income.drop('Description', axis=1, inplace=True)
UK_income.fillna(0, inplace=True)
UK_income.drop_duplicates(['SOC2010', 'Region'], inplace=True)
UK_income = UK_income.pivot('SOC2010', 'Region')
UK_income.columns.names = ['Features', 'Region']
UK_income = UK_income.swaplevel('Features', 'Region', axis=1)
UK_income['Universal', UK_income.index.name] = UK_income.index.map(int)

# merge in income, careful to exclude the aggregate rows that exist in the income spreadsheet
UK_to_ISCO = pd.merge(UK_to_ISCO, UK_income, how='inner')

UK_to_ISCO.rename(columns={'SOC2010': 'SOC\n2010'}, inplace=True)


In [278]:

UK_to_ISCO.head()

Region   Universal                                           \
Features    ISCO08                 ISCO08 Unit Group Titles   
0             1120  Managing directors and chief executives   
1             1111                              Legislators   
2             1321                   Manufacturing managers   
3             1323                    Construction managers   
4             1322                          Mining managers   

Region                                                                  \
Features                          SOC2010 Unit Group Titles Unnamed: 4   
0                     Chief executives and senior officials        NaN   
1                      Elected officers and representatives        NaN   
2         Production managers and directors in manufactu...        NaN   
3         Production managers and directors in construction        NaN   
4         Production managers and directors in mining an...        NaN   

Region           North East_x North West_x Merseyside_x  \
Features SOC2010   Employment   Employment   Employment   
0           1115          621         4241          827   
1           1116          269         1465            0   
2           1121        11260        31887         2704   
3           1122         6956        10549         3079   
4           1123          519            0            0   

Region   Yorkshire & Humberside_x East Midlands_x  \
Features               Employment      Employment   
0                            2556            4460   
1                               0            1098   
2                           24008           26571   
3                           13246           13447   
4                            1243               0   

Region                ...                                East Midlands  \
Features              ...               First degree/foundation degree   
0                     ...                                         1867   
1                     ...                                            0   
2                     ...                                         4910   
3                     ...                                         1457   
4                     ...                                            0   

Region                    West Midlands                        Eastern  \
Features First degree/foundation degree First degree/foundation degree   
0                                  1247                           2615   
1                                     0                              0   
2                                  8163                           6688   
3                                  1964                           3653   
4                                     0                              0   

Region                           London                     South East  \
Features First degree/foundation degree First degree/foundation degree   
0                                 10548                           4660   
1                                     0                           1223   
2                                  5456                           7276   
3                                  4954                           6358   
4                                     0                              0   

Region                       South West                          Wales  \
Features First degree/foundation degree First degree/foundation degree   
0                                  2494                            726   
1                                     0                              0   
2                                  5467                           2423   
3                                  1440                           1101   
4                                   803                              0   

Region                         Scotland               Northern Ireland  \
Features First degree/foundation degree First degree/foundation degree   
0                            